# Timestamp data

In [ ]:
# Imports
import pandas as pd
import numpy as np
from tqdm.auto import tqdm as tqdm
tqdm.pandas(desc="Progress")

In [6]:
# Metadata
header_rows = 3
input_file = "DATA.CSV"
output_file = "INFLUX_" + input_file

In [7]:
# Load data
df = pd.read_csv(input_file, header=header_rows, engine='pyarrow')

# Load data header
header = ""
with open(input_file) as file:
    for x in range(header_rows):
        header += next(file)

In [8]:
# Convert time to datetime dtype
df["_time"] = pd.to_datetime(df["_time"], unit="ms")

In [9]:
# Determine first timestamp
now = pd.Timestamp.now()
last = df["_time"].values[-1]
start = now - last

In [10]:
# Add first timestamp to time
df["_time"] += start

In [ ]:
# Format time as ISO string
df["_time"] = df["_time"].progress_apply(lambda x: x.isoformat())

In [ ]:
# Save data to CSV
df.to_csv(output_file, header=False, index=False, date_format=None)

In [ ]:
# Add header to CSV
with open(output_file, "r+") as file:
    content = file.read()
    file.seek(0, 0)
    file.write(header + content)

# Influxdb Client

In [ ]:
import os
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

token = os.environ.get("INFLUXDB_TOKEN")
org = "Synology"
bucket = "Arduino"

# with InfluxDBClient(url="http://ds220plus.local:8086", token=token, org=org) as client: